## Attribute access auditing

The `AttributeAuditor` stores all UUIDs of the `ModelObject`s on which specified (or any) attributes where called during the life of the `MelodyModel`. 

In [21]:
import capellambse

path_to_model = "../../../tests/data/model/RM Bridge.aird"
model = capellambse.MelodyModel(path_to_model, jupyter_untrusted=True)

Cannot load PVMT extension: ValueError: Provided model does not have a PropertyValuePkg
Property values are not available in this model


## Change auditing

The `ChangeAuditor` stores the context of all changes (modifications, extensions and deletions) grouped by its UUID on `ModelObject`s with matching type to the specified class-names. 

In [22]:
from rm_bridge import auditing

req = model.oa.all_requirements.by_name("TestReq1", single=True)
other_req = model.oa.all_requirements[1]
with auditing.ChangeAuditor(model) as changes:
    print(req.uuid)
    req.long_name = "Not TestReq anymore"
    req.attributes.insert(0, other_req.attributes[0])
    del req.attributes[-1]

3c2d312c-37c9-41b5-8c32-67578fa52dc3


In [23]:
import pprint

pretty_printer = pprint.PrettyPrinter()

pretty_printer.pprint(changes)

[Modification(parent='3c2d312c-37c9-41b5-8c32-67578fa52dc3',
              attribute='long_name',
              new='Not TestReq anymore',
              old='1'),
 Extension(parent='3c2d312c-37c9-41b5-8c32-67578fa52dc3',
           attribute='attributes',
           element='<EnumerationValueAttribute [Enumeration Value Attribute] '
                   "'' (148bdf2f-6dc2-4a83-833b-596886ce5b07)>",
           uuid='148bdf2f-6dc2-4a83-833b-596886ce5b07'),
 Deletion(parent='3c2d312c-37c9-41b5-8c32-67578fa52dc3',
          attribute='attributes',
          element='<BooleanValueAttribute [Boolean Value Attribute] False '
                  '(dcb8614e-2d1c-4cb3-aa0c-667a297e7489)>',
          uuid='dcb8614e-2d1c-4cb3-aa0c-667a297e7489')]


### Filtering of change-events work via passed class-names

In [24]:
req_module = model.by_uuid("f8e2195d-b5f5-4452-a12b-79233d943d5e")
req = req_module.requirements[0]
with auditing.ChangeAuditor(model, {"RequirementsModule"}) as changes:
    print(f"<{req_module.long_name} ({req_module.uuid})")
    req_module.long_name = "Not Module anymore"
    
    req.long_name = "Doesn't show up"
    print(f"<{req.long_name} ({req.uuid})")

<Module (f8e2195d-b5f5-4452-a12b-79233d943d5e)
<Doesn't show up (85d41db2-9e17-438b-95cf-49342452ddf3)


In [25]:
pretty_printer.pprint(changes)

[Modification(parent='f8e2195d-b5f5-4452-a12b-79233d943d5e',
              attribute='long_name',
              new='Not Module anymore',
              old='Module')]


In [26]:
import yaml

print(yaml.dump(changes, indent=4))

- !!python/object:rm_bridge.auditing.Modification
    attribute: long_name
    new: Not Module anymore
    old: Module
    parent: f8e2195d-b5f5-4452-a12b-79233d943d5e



### Writing the change-context to a file

Here the `safe_dump` option comes in handy:

In [27]:
import json

from capella_rm_bridge import auditing

req = model.oa.all_requirements.by_name("TestReq1")
with auditing.ChangeAuditor(model) as changes:
    req.long_name = "Changed Name"
    req.attributes.create("enum")
    del req.attributes[0]

dumpable = auditing.dump(changes)
print(json.dumps(dumpable, indent=4))
print(yaml.dump(dumpable, indent=2))

[
    {
        "_type": "Modification",
        "parent": "3c2d312c-37c9-41b5-8c32-67578fa52dc3",
        "attribute": "long_name",
        "new": "Changed Name",
        "old": "Not TestReq anymore"
    },
    {
        "_type": "Extension",
        "parent": "3c2d312c-37c9-41b5-8c32-67578fa52dc3",
        "attribute": "attributes",
        "element": "<EnumerationValueAttribute [Enumeration Value Attribute] '' (15df41af-f2e8-45b5-9d80-882bd45905f5)>",
        "uuid": "15df41af-f2e8-45b5-9d80-882bd45905f5"
    },
    {
        "_type": "Deletion",
        "parent": "3c2d312c-37c9-41b5-8c32-67578fa52dc3",
        "attribute": "attributes",
        "element": "<EnumerationValueAttribute [Enumeration Value Attribute] '' (148bdf2f-6dc2-4a83-833b-596886ce5b07)>",
        "uuid": "148bdf2f-6dc2-4a83-833b-596886ce5b07"
    }
]
- _type: Modification
  attribute: long_name
  new: Changed Name
  old: Not TestReq anymore
  parent: 3c2d312c-37c9-41b5-8c32-67578fa52dc3
- _type: Extension
  attrib